In [2]:
import pandas as pd
import random
import time
import numpy as np
import timeit
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import xgboost as xgb
from sklearn.model_selection import train_test_split

C:\Users\Nick\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
train_data = pd.read_csv("sales_train.csv")
items = pd.read_csv("items.csv")
shops = pd.read_csv("shops.csv")
test_data = pd.read_csv('test.csv')

In [ ]:
plt.boxplot(train_data["item_cnt_day"])
plt.show()
print(train_data.describe())

In [4]:
def rem_err_values(dataset):
    """
    Remove all values that are considered to be erroneous.
    """
    dataset = dataset[dataset["item_cnt_day"] > 0] #you cannot sell negative number of items
    return dataset

def find_outliers_iforrest(dataset):
    """Take one argument (pd.dataframe) and return this dataframe without the outliers.
    Removes column "date" in the process.
    """
    start = time.time()
    dataset = dataset.drop(columns="date")
    dataset = rem_err_values(dataset)
    iso = IsolationForest()
    outlier_model = iso.fit_predict(dataset)
    mask = outlier_model != -1
    dataset = dataset.loc[mask]
    print(f'Walltime: {time.time()-start}')
    return dataset
    
def find_outliers_IQR(dataset, column):
    dataset = dataset.drop(columns="date")
    Q1 = dataset[column].quantile(0.25)
    Q3 = dataset[column].quantile(0.75)
    IQR = Q3 - Q1
    dataset = dataset[dataset[column]<(Q3+1.5*IQR)]
    return dataset

In [5]:
print(f'Missing values per group:\n{df_test.isna().sum()}\n')
def correlation(dataset):
    corr = dataset.corr()
    corr = corr[corr<1]
    corr = abs(corr)
    print(f'{corr}\n')
    corr = corr.loc[["item_cnt_day"]]
    print(f'Percentage correlation for item_cnt_day:\n{corr.max()*100}')
correlation(df_test)

NameError: name 'df_test' is not defined

In [ ]:
def linreg(dataset):
    X = dataset[["item_id"]]
    y = dataset[["item_cnt_day"]]
    reg = LinearRegression().fit(X, y)
    pred = reg.predict(X)
    rms = mean_squared_error(y, pred, squared=False)
    return rms
    
linreg(train_data)

In [5]:
def DataPrep(items, train_data):
    items = items.drop(['item_name'], axis = 1)
    df = pd.merge(items, train_data, how = 'inner')
    df = df.drop(['date', 'item_price'], axis = 1)
    temp_data = df.groupby(['date_block_num','shop_id','item_id','item_category_id']).sum().reset_index()
    temp_data.rename(columns = {'item_cnt_day' : 'item_cnt_month'},inplace=True)
    temp_data = temp_data.pivot_table(index=['shop_id','item_id', 'item_category_id'], columns='date_block_num', values='item_cnt_month').fillna(0.0)
    return temp_data

In [7]:
#######################################################
xgboost

NameError: name 'xgboost' is not defined

In [8]:
train_data = rem_err_values(train_data)
clean_data = DataPrep(items, train_data)
clean_data.head()

date_block_num                     0     1    2    3    4    5    6    7   \
shop_id item_id item_category_id                                            
0       30      40                0.0  31.0  0.0  0.0  0.0  0.0  0.0  0.0   
        31      37                0.0  11.0  0.0  0.0  0.0  0.0  0.0  0.0   
        32      40                6.0  10.0  0.0  0.0  0.0  0.0  0.0  0.0   
        33      37                3.0   3.0  0.0  0.0  0.0  0.0  0.0  0.0   
        35      40                1.0  14.0  0.0  0.0  0.0  0.0  0.0  0.0   

date_block_num                     8    9   ...   24   25   26   27   28   29  \
shop_id item_id item_category_id            ...                                 
0       30      40                0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0   
        31      37                0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0   
        32      40                0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0   
        33      37                0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0   
        35      40                0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0   

date_block_num                     30   31   32   33  
shop_id item_id item_category_id                      
0       30      40                0.0  0.0  0.0  0.0  
        31      37                0.0  0.0  0.0  0.0  
        32      40                0.0  0.0  0.0  0.0  
        33      37                0.0  0.0  0.0  0.0  
        35      40                0.0  0.0  0.0  0.0  

[5 rows x 34 columns]

In [9]:
X = clean_data.iloc[:,  (clean_data.columns != 33)].values
y = clean_data.iloc[:, clean_data.columns == 33].values

xgr = xgb.XGBRegressor()
xgr.fit(X,y, verbose=True)
preds = xgr.predict(X)
rmse = np.sqrt(mean_squared_error(preds,y))
print(rmse)

1.0658452600970079


In [8]:
submit_df = test_data.merge(clean_data, how = "left", on = ["shop_id", "item_id"]).fillna(0.0)
ids = submit_df.pop('ID')
submit_df.set_index(['shop_id', 'item_id'], inplace=True)
submit_df = submit_df.iloc[:,1:]
submit_df.head()

1    2    3    4    5    6    7    8    9    10  ...   24  \
shop_id item_id                                                    ...        
5       5037     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  2.0   
        5320     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
        5233     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
        5232     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
        5268     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

                  25   26   27   28   29   30   31   32   33  
shop_id item_id                                               
5       5037     0.0  0.0  0.0  1.0  1.0  1.0  3.0  1.0  0.0  
        5320     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
        5233     0.0  0.0  0.0  3.0  2.0  0.0  1.0  3.0  1.0  
        5232     0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  
        5268     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 33 columns]

In [9]:
preds = xgr.predict(submit_df.values)
preds2 = pd.DataFrame(preds)

In [10]:
mmscaler = MinMaxScaler(feature_range=(0, 20))
preds2 = pd.DataFrame(mmscaler.fit_transform(preds2))
sub_df = pd.DataFrame({'ID':ids.values, 'item_cnt_month':preds2[0].values})
print(sub_df)

            ID  item_cnt_month
0            0        0.484029
1            1        0.481201
2            2        0.522264
3            3        0.473489
4            4        0.481201
...        ...             ...
214195  214195        0.470857
214196  214196        0.481201
214197  214197        0.445694
214198  214198        0.481201
214199  214199        0.455242

[214200 rows x 2 columns]


In [11]:
sub_df.to_csv("submission_test_xgb.csv", index=False)

In [ ]:
#####################################################
#xgb right way

In [37]:
def DataPrep2(items, train_data):
    items = items.drop(['item_name'], axis = 1)
    df = pd.merge(items, train_data, how = 'inner', on='item_id')
    df = df.drop(['date'], axis = 1)
    temp_data = df.groupby(['date_block_num','item_category_id', 'shop_id', 'item_price', 'item_id']).sum().reset_index()
    temp_data.dropna(inplace=True)
    temp_data.rename(columns = {'item_cnt_day' : 'item_cnt_month'},inplace=True)
    temp_data.drop(['item_category_id', 'item_price'], axis=1, inplace=True)
    X = temp_data.copy()
    y = X.pop('item_cnt_month')
    y = pd.DataFrame(y)
    X_train = X[X.date_block_num != 33]
    X_test = X[X.date_block_num == 33]
    arr = X['date_block_num']
    y.insert(0, "date_block_num", arr)
    y_train = y[y.date_block_num != 33]
    y_test = y[y.date_block_num == 33]
    y_train.drop('date_block_num', axis=1, inplace=True)
    y_test.drop('date_block_num', axis=1, inplace=True)
    return X_train, X_test, y_train, y_test

In [38]:
X_train, X_test, y_train, y_test = DataPrep2(items, train_data)
# X_test = X_test['date_block_num']
print(X_train)

         date_block_num  shop_id  item_id
0                     0       25    16257
1                     0       45     5606
2                     0        0     5623
3                     0        0     5573
4                     0        0     5575
...                 ...      ...      ...
1705819              32       58    22091
1705820              32       58    22092
1705821              32       59    22087
1705822              32       59    22088
1705823              32       59    22091

[1705824 rows x 3 columns]


C:\Users\Nick\AppData\Local\Temp\ipykernel_18312\590548710.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train.drop('date_block_num', axis=1, inplace=True)
C:\Users\Nick\AppData\Local\Temp\ipykernel_18312\590548710.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test.drop('date_block_num', axis=1, inplace=True)


In [143]:
scaler = StandardScaler()
arr = scaler.fit_transform(X_train[['item_price']]).flatten()
X_train['item_price'] = arr
arr = scaler.transform(y_train[['item_price']]).flatten()
y_train['item_price'] = arr
y_train

KeyError: "None of [Index(['item_price'], dtype='object')] are in the [columns]"

In [23]:
xgr = xgb.XGBRegressor()
xgr.fit(X_train, y_train)
preds = xgr.predict(pd.DataFrame(X_test))
rmse = np.sqrt(mean_squared_error(preds,y_test))
print(rmse)

ValueError: Feature shape mismatch, expected: 4, got 1

13.143200939964048


In [ ]:
#######################################################
# Lin regression

In [9]:
train_data = rem_err_values(train_data)
clean_data = DataPrep(items, train_data)

X = clean_data.iloc[:,  (clean_data.columns != 33)].values
y = clean_data.iloc[:, clean_data.columns == 33].values

In [20]:
lin_clf = LinearRegression()
regressor = lin_clf.fit(X, y)
pred = regressor.predict(X)
rmse = np.sqrt(mean_squared_error(pred,y))
print(rmse)

preds = 

3.1182742091114872


In [23]:
def get_data(test_data, clean_data, model):
    submit_df = test_data.merge(clean_data, how = "left", on = ["shop_id", "item_id"]).fillna(0.0)
    ids = submit_df.pop('ID')
    submit_df.set_index(['shop_id', 'item_id'], inplace=True)
    submit_df = submit_df.iloc[:,1:]
    preds = model.predict(submit_df.values)
    preds2 = pd.DataFrame(preds)
    mmscaler = MinMaxScaler(feature_range=(0, 20))
    preds2 = pd.DataFrame(mmscaler.fit_transform(preds2))
    sub_df = pd.DataFrame({'ID':ids.values, 'item_cnt_month':preds2[0].values})
    return sub_df

In [30]:
df_submssion = get_data(test_data, clean_data, regressor)

ID                214199.0
item_cnt_month        20.0
dtype: float64


In [31]:
df_submssion.to_csv('submission_linear.csv', index=False)